In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
RANDOM_STATE = 5
DATA_NAME = "ReCo"

In [ ]:
import get_data
df_obs,descriptor_names, target_name = get_data.load(DATA_NAME)
Xraw = df_obs.loc[:, descriptor_names].values
y = df_obs.loc[:, target_name].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xraw)
X = scaler.transform(Xraw)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
# データからの学習
nfold = 10
kf = KFold(nfold, shuffle=True, random_state=RANDOM_STATE)
estimator = KernelRidge(alpha=1, gamma=1, kernel="rbf")
param_grid = {"alpha": np.logspace(-6, 0, 11), 
              "gamma": np.logspace(-5, 0, 11)}
reg_cv = GridSearchCV(estimator,  cv=kf, param_grid=param_grid, scoring='r2') #, verbose=2)
reg_cv.fit(X, y)
print("best hyperparameter")
print(reg_cv.best_params_)
print("best score")
print(reg_cv.best_score_)

In [ ]:
from regression_misc import plot_y_yp
from sklearn.metrics import r2_score
yp = reg_cv.predict(X)
# R2の評価
kr_score = r2_score(y, yp)
print("R2=", kr_score)
plot_y_yp(y, yp)


In [ ]:
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import make_scorer
kf = KFold(nfold, shuffle=True, random_state=RANDOM_STATE)
reg = KernelRidge(alpha=reg_cv.best_params_["alpha"], 
                  gamma=reg_cv.best_params_["gamma"], kernel="rbf")

cv_results = cross_validate(reg, X, y, scoring=make_scorer(r2_score), cv=kf)
print(np.mean(cv_results["test_score"]), np.std(cv_results["test_score"]))

kf = KFold(nfold, shuffle=True, random_state=RANDOM_STATE)
yp = cross_val_predict(reg, X, y, cv=kf)
plot_y_yp(y, yp)